In [1]:
import os
from os.path import join as pjoin, exists as pexists, abspath, realpath
import logging
import sqlite3

from sqlite3 import PARSE_DECLTYPES, PARSE_COLNAMES
from Utilities.singleton import Singleton
from Utilities.process import pGetProcessedFiles, pAlreadyProcessed, pWriteProcessedFile
from Utilities.files import flGetStat

from datetime import datetime
import unicodedata
import re

import numpy as np
from numpy.lib.recfunctions import append_fields

import warnings
warnings.filterwarnings('error')

In [2]:
TBLLOCATIONSDEF = ("CREATE TABLE IF NOT EXISTS tblLocations "
                   "(stnId integer PRIMARY KEY, stnName text, stnType text, "
                   "stnDistCode text, stnWMONumber integer, "
                   "stnLon real, stnLat real, stnElevation real, "
                   "stnDateOpen timestamp, stnDateClosed timestamp, "
                   "stnDataStart integer, stnDataEnd integer, "
                   "dtCreated timestamp, dtUpdated timestamp)")

TBLWXDESCDEF = ("CREATE TABLE IF NOT EXISTS tblWxDesc "
                "(stnId integer, dtLocalTime timestamp, dtLocalStdTime timestamp, "
                "stnPresentWxCode integer, stnPresentWxDesc text, "
                "stnPresentWxQual text, stnPastWxCode int, stnPastWxDesc text, "
                "stnPastWxQual text, dtCreated timestamp, dtUpdated timestamp)")

TBLDAILYMAXGUSTDEF = ("CREATE TABLE IF NOT EXISTS tblDailyMaxGust "
                      "(stnId integer, dtLocalTime timestamp, stnGustSpeed real, "
                      "stnGustSpeedUnits text, stnGustSpeedQual text, stnGustDirection real, "
                      "stnGustDirQual text, stnGustTimeQual text, dtCreated timestamp, dtUpdated timestamp)")

CONFIGDEFAULTS = """
[Input]
StationFile=C:/WorkSpace/data/Raw/obs/weather/HC06D_StnDet_99999999720737.txt

[Weather]
Path=C:/WorkSpace/data/Raw/obs/weather/
Basename=HC06D_Data_{0:06d}_99999999720737.txt

[MaxGust]
Path=C:/WorkSpace/data/Raw/obs/daily/
Basename=DC02D_Data_{0:06d}_99999999720437.txt

[Process]
DatFile=C:/WorkSpace/data/Raw/obs/processed_obs_files.dat
ExcludePastProcessed=True

[Database]
Path=C:/WorkSpace/data/Raw/obs/

[Logging]
LogFile=C:/WorkSpace/data/Raw/obs/process_obs_files.log
LogLevel=INFO
Verbose=True
Timestamp=True
"""

In [3]:
INSLOCATIONS = ("INSERT OR REPLACE INTO tblLocations "
                "VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)")

INSWXDESC = ("INSERT INTO tblWxDesc "
             "VALUES (?,?,?,?,?,?,?,?,?,?,?)")

INSMAXGUST = ("INSERT INTO tblDailyMaxGust "
              "VALUES (?,?,?,?,?,?,?,?,?,?)")

# Update a location selected by stnId:
# Need to set {colname} before execution
UPDATELOC = ("UPDATE tblLocations SET {colname} = ?, dtUpdated = ? "
             "WHERE stnId = ?")

In [4]:
pastWxCode = {0:"Cloud covering half or less of the sky throughout the period",
              1:"Cloud covering more than half the sky during part of the period & half or less for the rest",                                              
              2:"Cloud covering more than half the sky throughout the period",
              3:"Sandstorm, duststorm or blowing snow",
              4:"Fog or ice fog or thick haze",
              5:"Drizzle",
              6:"Rain1",
              7:"Snow, or rain and snow mixed",
              8:"Shower(s)",
              9:"Thunderstorm(s) with or without precipitation"}

presentWxCode = {0:"Cloud development not observed or not observable",
                 1:"Cloud generally dissolving or becoming less developed",
                 2:"State of sky on the whole unchanged",
                 3:"Clouds generally forming or developing",
                 4:"Visibility reduced by smoke, e.g. veldt or forest fires, industrial smoke or volcanic ashes",
                 5:"Haze",
                 6:"Widespread dust in suspension in the air, not raised by wind at or near the station at the time of observation",
                 7:"Dust or sand raised by wind at or near the station at the time of observation, but not well-developed dust whirl(s) or sand whirl(s), and no duststorm or sandstorm seen; or, in the case of ships, blowing spray at the station",
                 8:"Well-developed dust or sand whirl(s) seen at or near the station during the preceding hour or at the time of observation, but no dust storm or sandstorm",
                 9:"Duststorm or sandstorm within sight at the time of observation, or at the station during the preceding hour",
                 10:"Mist",
                 11:"Patches of shallow fog or ice fog at the station, whether on land or sea not deeper than about 2 metres on land or 10 metres at sea",
                 12:"More or less continuous shallow fog or ice fog at the station, whether on land or sea, not deeper than about 2m/land or 10m/sea",
                 13:"Lightning visible, no thunder heard",
                 14:"Precipitation within sight, not reaching the ground or the surface of the sea",
                 15:"Precipitation within sight, reaching the ground or the surface of the sea, but distant, i.e. > 5 km from the station",
                 16:"Precipitation within sight, reaching the ground or the surface of the sea, near to, but not at the station",
                 17:"Thunderstorm, but no precipitation at the time of observation",
                 18:"Squalls at or within sight of the station during the preceding hour or at the time of observation",
                 19:"Funnel clouds at or within sight of the station during the preceding hour or at the time of observation",
                 20:"Drizzle (not freezing) or snow grains, not falling as showers, during the preceding hour but not at the time of observation",
                 21:"Rain (not freezing), not falling as showers, during the preceding hour but not at the time of observation",
                 22:"Snow, not falling as showers, during the preceding hour but not at the time of observation",
                 23:"Rain and snow or ice pellets, not falling as showers; during the preceding hour but not at the time of observation",
                 24:"Freezing drizzle or freezing rain; during the preceding hour but not at the time of observation",
                 25:"Shower(s) of rain during the preceding hour but not at the time of observation",
                 26:"Shower(s) of snow, or of rain and snow during the preceding hour but not at the time of observation",
                 27:"Shower(s) of hail, or of rain and hail during the preceding hour but not at the time of observation",
                 28:"Fog or ice fog during the preceding hour but not at the time of observation",
                 29:"Thunderstorm (with or without precipitation) during the preceding hour but  not at the time of observation",
                 30:"Slight or moderate duststorm or sandstorm - has decreased during the preceding hour",
                 31:"Slight or moderate duststorm or sandstorm - no appreciable change during the preceding hour",
                 32:"Slight or moderate duststorm or sandstorm - has begun or has increased during the preceding hour",
                 33:"Severe duststorm or sandstorm - has decreased during the preceding hour",
                 34:"Severe duststorm or sandstorm - no appreciable change during the preceding hour",
                 35:"Severe duststorm or sandstorm - has begun or has increased during the preceding hour",
                 36:"Slight/moderate drifting snow - generally low (below eye level)",
                 37:"Heavy drifting snow - generally low (below eye level)",
                 38:"Slight/moderate blowing snow - generally high (above eye level)",
                 39:"Heavy blowing snow - generally high (above eye level)",
                 40:"Fog or ice fog at a a distance at the time of observation, but not at station during the preceding hour, the fog or ice fog extending to a level above that of  the observer",
                 41:"Fog or ice fog in patches",
                 42:"Fog/ice fog, sky visible, has become thinner during the preceding hour",
                 43:"Fog/ice fog, sky invisible, has become thinner during the preceding hour",
                 44:"Fog or ice fog, sky visible, no appreciable change during the past hour",
                 45:"Fog or ice fog, sky invisible, no appreciable change during the preceding hour",
                 46:"Fog or ice fog, sky visible, has begun or has become thicker during preceding hour",
                 47:"Fog or ice fog, sky invisible, has begun or has become thicker during the preceding hour",
                 48:"Fog, depositing rime, sky visible",
                 49:"Fog, depositing rime, sky invisible",
                 50:"Drizzle, not freezing, intermittent, slight at time of ob.",
                 51:"Drizzle, not freezing, continuous, slight at time of ob.",
                 52:"Drizzle, not freezing, intermittent, moderate at time of ob.",
                 53:"Drizzle, not freezing, continuous, moderate at time of ob.",
                 54:"Drizzle, not freezing, intermittent, heavy at time of ob.",
                 55:"Drizzle, not freezing, continuous, heavy at time of ob.",
                 56:"Drizzle, freezing, slight",
                 57:"Drizzle, freezing, moderate or heavy (dense)",
                 58:"Rain and drizzle, slight",
                 59:"Rain and drizzle, moderate or heavy",
                 60:"Rain, not freezing, intermittent, slight at time of ob.",
                 61:"Rain, not freezing, continuous, slight at time of ob.",
                 62:"Rain, not freezing, intermittent, moderate at time of ob.",
                 63:"Rain, not freezing, continuous, moderate at time of ob.",
                 64:"Rain, not freezing, intermittent, heavy at time of ob.",
                 65:"Rain, not freezing, continuous, heavy at time of ob.",
                 66:"Rain, freezing, slight",
                 67:"Rain, freezing, moderate or heavy",
                 68:"Rain or drizzle and snow, slight",
                 69:"Rain or drizzle and snow, moderate or heavy",
                 70:"Intermittent fall of snowflakes, slight at time of ob.",
                 71:"Continuous fall of snowflakes, slight at time of ob.",
                 72:"Intermittent fall of snowflakes, moderate at time of ob.",
                 73:"Continuous fall of snowflakes, moderate at time of ob.",
                 74:"Intermittent fall of snowflakes, heavy at time of ob.",
                 75:"Continuous fall of snowflakes, heavy at time of ob.",
                 76:"Diamond dust (with or without fog)",
                 77:"Snow grains (with or without fog)",
                 78:"Isolated star-like snow crystals (with or without fog)",
                 79:"Ice pellets",
                 80:"Rain shower(s), slight",
                 81:"Rain shower(s), moderate or heavy",
                 82:"Rain shower(s), violent",
                 83:"Shower(s) of rain and snow, slight",
                 84:"Shower(s) of rain and snow, moderate or heavy",
                 85:"Snow shower(s), slight",
                 86:"Snow shower(s), moderate or heavy",
                 87:"Shower(s) of snow pellets or small hail, with or without rain or rain and snow mixed - slight",
                 88:"Shower(s) of snow pellets or small hail, with or without rain or rain and snow mixed - moderate or heavy",
                 89:"Shower(s) of hail, with or without rain or rain and snow mixed, not associated with thunder - slight",
                 90:"Shower(s) of hail, with or without rain or rain and snow mixed, not associated with thunder - moderate or heavy",
                 91:"Slight rain at time of observation - Thunderstorm during the preceding hour but not at time of observation",
                 92:"Moderate or heavy rain at time of observation - Thunderstorm during the preceding hour but not at time of observation",
                 93:"Slight snow, or rain and snow mixed or hail at time of observation - Thunderstorm during the preceding hour but not at time of observation",
                 94:"Moderate or heavy snow, or rain and snow mixed or hail at time of observation - Thunderstorm during the preceding hour but not at time of observation",
                 95:"Thunderstorm, slight or moderate, without hail, but with rain and/or snow at time of observation",
                 96:"Thunderstorm, slight or moderate, with hail at time of observation",
                 97:"Thunderstorm, heavy, without hail, but with rain and/or snow at time of observation",
                 98:"Thunderstorm combined with dust/sandstorm at time of observation",
                 99:"Thunderstorm, heavy with hail at time of observation"
    }

maxlen = 0
for k, v in presentWxCode.items():
    if len(v) > maxlen:
        maxlen = len(v)
        print "Longest code description is code {0} ({1} characters): \n{2}".format(k, len(v), v)

In [5]:
def parseTimeString(yr, month, day, timestr):
    """
    Parse year, month and day as strings and return a datetime.
    
    Handles the case of a missing time string (Pandas returns nan 
    if the field is empty).
    """
    if timestr is np.nan:
        timetrs='0000'
    formattime = '{0}-{1}-{2} {3}'.format(yr, month, day, time)
    
    return datetime.strptime(formattime, '%Y-%m-%d %H%M')

def convertStartDate(datestr):
    if datestr == '':
        return ""
    dtstr = "01/{0}".format(datestr)
    return datetime.strptime(dtstr, '%d/%m/%Y')

def parseObsDate(year, month, day, hour, minute):
    timestr = "{0}-{1}-{2} {3}:{4}".format(year, month, day, hour, minute)
    return datetime.strptime(timestr, '%Y-%m-%d %H:%M')

In [6]:
LOCTYPES = [('st', 'S2'), ('stnId', 'i'), ('stnDistCode', 'S4'), ('stnName', 'S40'), 
            ('stnDateOpen', 'S10'), ('stnDateClosed', 'S10'), ('stnLat', 'f8'), 
            ('stnLon', 'f8'), ('method', 'S15'), ('state', 'S3'), 
            ('stnElevation', 'f8'), ('baroElev', 'i'), ('stnWMONumber', 'i'), ('stnDataStart', 'i'), 
            ('stnDataEnd', 'i'), ('blank', 'S3'), ('percentcomplete', 'f8'), ('pcqualy', 'f8'), 
            ('pcqualn', 'f8'), ('pcqualw', 'f8'), ('pcquals', 'f8'), ('pcquali', 'f8'), ('end', 'S1')]
LOCCONVERT = {4: lambda s: convertStartDate(s),
              5: lambda s: convertStartDate(s)}

DMAXTYPES = [('dc', 'S2'), ('stnId' ,'i'), ('Year', 'i'), ('Month', 'i'),
             ('Day', 'i'), ('stnGustSpeed', 'f'), ('stnGustSpeedQual', 'S1'),
             ('stnGustDir', 'f8'), ('stnGustDirQual', 'S1'), 
             ('stnGustTime', 'S4'), ('stnGustTimeQual', 'S1'), ('end', 'S1')]

DAILYMAXCONVERT = {'stnGustSpeed': lambda s: float(s or 0)}

WXDESCTYPES = [('hc','S2'), ('stnId', 'i'), 
               ('dtLocalYear', 'i'), ('dtLocalMonth', 'i'), ('dtLocalDay', 'i'), ('dtLocalHour', 'i'), ('dtLocalMin', 'i'), 
               ('dtStdYear', 'i'), ('dtStdMonth', 'i'), ('dtStdDay', 'i'), ('dtStdHour', 'i'), ('dtStdMin', 'i'), 
               ('stnPresentWxCode', 'i'), ('stnPresentWxCodeQual', 'S1'), 
               ('stnPastWxCode', 'i'), ('stnPastWxCodeQual', 'S1'), ('end', 'S1')]
WXDESCCONVERT = {2: lambda s: convertObsDate(s),
                 3: lambda s: convertObsDate(s)}


In [7]:
locfile = "C:/WorkSpace/data/Raw/obs/weather/HC06D_StnDet_99999999720737.txt"
stndata = np.genfromtxt(locfile, dtype=LOCTYPES, delimiter=",", 
                        usecols=(1,2,3,4,5,6,7,9,10,12,13,14,16), 
                        converters=LOCCONVERT, autostrip=True, skip_footer=1)

In [8]:
gustfile = "C:/WorkSpace/data/Raw/obs/daily/DC02D_Data_200283_99999999720437.txt"
dailydata = np.genfromtxt(gustfile, dtype=DMAXTYPES, delimiter=",",
                         usecols=(1,2,3,4,5,6,7,8,9),
                         autostrip=True, skip_header=1, skip_footer=1)

In [9]:
dailydata

In [10]:
wxfile = "C:/WorkSpace/data/Raw/obs/weather/HC06D_Data_200786_99999999720737.txt"
wxdata = np.genfromtxt(wxfile, dtype=WXDESCTYPES, delimiter=",",
                         usecols=(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15),
                         autostrip=True, skip_header=1, skip_footer=1)

In [11]:
wxdata

In [13]:


wxdatapath =  "C:/WorkSpace/data/Raw/obs/weather/"
basename = "HC06D_Data_{0:06d}_99999999720737.txt" 

for stn in stndata['stnId']:
    fname = os.path.join(wxdatapath, basename.format(int(stn)))
    if os.path.exists(fname):
        try:
            wxdata = np.genfromtxt(fname, dtype=WXDESCTYPES, delimiter=",",
                                   usecols=(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15),
                                   autostrip=True, skip_header=1, skip_footer=1)
        except UserWarning:
            print "{0} is empty".format(fname)
    else:
        print "{0} does not exist".format(fname)

In [15]:
class ObsDatabase(sqlite3.Connection, Singleton):
    ignoreSubsequent = True
    def __init__(self, configFile):
        
        self.config = ConfigParser(defaults=CONFIGDEFAULTS)
        self.config.read(configFile)
        self.dbpath = self.config.read("Database", "Path")
        self.obsDB = pjoin(self.dbpath, "observations.db")
        self.datfile = self.config.get("Process", "DatFile")
        self.excludePastProcessed = self.config.getboolean("Process", "ExcludePastProcessed")
        pGetProcessedFiles(self.datfile)
        sqlite3.Connection.__init__(self, self.obsDB, detect_types=PARSE_DECLTYPES|PARSE_COLNAMES)
        
    def createDatabase(self):
        """
        Create the observation database, using the configuration options
        specified in the configuration file.
        """
        
        self.createTable('tblLocations', TBLLOCATIONSDEF)
        self.createTable('tblDailyMaxGust', TBLDAILYMAXGUSTDEF)
        self.createTable('tblWxDesc', TBLWXDESCDEF)
        self.commit()
        return
    
    def createTable(self, tblName, tblDef):
        """
        Create a table based on the table definition.
        
        :param str tblName: Name of the table to be created.
        :param str tblDef: SQL table defintion statement.
        """
        try:
            self.execute(tblDef)
            self.commit()
        except sqlite3.Error as err:
            log.exception("Cannot create table {0}: {1}".\
                          format(tblName, err.args[0]))
            raise

    def insertLocations(self):
        locations = []
        stnFile = self.config.get("Input", "StationFile")
        stndata = np.genfromtxt(stnFile, dtype=LOCTYPES, delimiter=",", 
                        usecols=(1,2,3,4,5,6,7,9,10,12,13,14,16), 
                        converters=LOCCONVERT, autostrip=True, skip_footer=1)
        dtCreated = datetime.now()
        dtUpdated = datetime.now()
        for rec in stndata:
            params = (rec['stnId'], rec['stnName'], rec['stnType'], rec['stnDistCode'], rec['stnWMONumber'],
                      rec['stnLon'], rec['stnLat'], rec['stnElevation'], rec['stnDateOpen'], rec['stnDateClosed'],
                      rec['stnDataStart'], rec['stnDataEnd'], dtCreated, dtUpdated)
            locations.append(params)
        self.executemany(INSLOCATIONS, locations)
        self.commit()
        return
    
    def getStationIdList(self):
        try:
            cur = self.execute(("SELECT stnId from tblLocations"))
        except sqlite3.Error as err:
            log.exception("Cannot retrieve station id's from tblLocations: {0}".\
                          format(err.args[0]))
            raise
        else:
            recs = cur.fetchall()
        stnids = np.rec.fromrecords(recs, names=('stnId',))
        return stnids
    
    def insertWxDesc(self, data):
        records = []
        dtCreated = datetime.now()
        dtUpdated = datetime.now()
        for rec in data:
            localTime = parseObsDate(rec['dtLocalYear'], rec['dtLocalMonth'], 
                                     rec['dtLocalDay'], rec['dtLocalHour'], 
                                     rec['dtLocalMin'])
            localStdTime = parseObsDate(rec['dtStdYear'], rec['dtStdMonth'], 
                                        rec['dtStdDay'], rec['dtStdHour'], 
                                        rec['dtStdMin'])
            presentWxDesc = presentWxCode[rec['stnPresentWxCode']]
            pastWxDesc = pastWxCode[rec['stnPastWxCode']]
            
            params = (rec['stnId'], localTime, localStdTime, 
                      rec['stnPresentWxCode'], presentWxDesc, rec['stnPresentWxCodeQual'],
                      rec['stnPastWxCode'], pastWxDesc, rec['stnPastWxCodeQual'],
                      dtCreated, dtUpdated)
            records.append(param)

        self.executemany(INSWXDESC, records)
        self.commit()
        
        return
    
    def insertMaxGustData(self, data): 
        # TODO: Fix setting of units! Presently hard coded to m/s
        records = []
        dtCreated = datetime.now()
        dtUpdated = datetime.now()
        for rec in data:
            stnGustTime = parseTimeStr(rec['Year'], rec['Month'], rec['Day'], rec['stnGustTime'])
            params = (rec['stnId'], stnGustTime, rec['stnGustSpeed'],
                      'm/s', rec['stnGustSpeedQual'], rec['stnGustDirection'],
                      rec['stnGustDirQual'], rec['stnGustTimeQual'], dtCreated, dtUpdated)
            records.append(params)
        self.executemany(INSMAXGUST, records)
        self.commit()
        return
    
    def processWxDescFile(self, wxdescfile):
        try:
            wxdata = np.genfromtxt(wxdescfile, dtype=WXDESCTYPES, delimiter=",",
                                   usecols=(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15),
                                   autostrip=True, skip_header=1, skip_footer=1)
        except UserWarning:
            log.info("{0} is an empty file".format(wxdescfile))
            return False
        else:
            self.insertWxDesc(wxdata)
            return True
        
    def processWeatherDescFiles(self):
        wxdatapath =  self.config.get("Weather", "Path")
        basename = self.config.get("Weather", "Basename") # 

        stations = self.getStationIdList()
        for stn in stations:
            fname = pjoin(wxdatapath, basename.format(int(stn)))
            if os.path.exists(fname):
                fstat = flGetStat(fname)
                if (pAlreadyProcessed(fstat[0], fstat[1], 'md5sum', fstat[2]) and
                    self.excludePastProcessed):
                    log.debug("Already processed {0}".format(fname))
                else:
                    if self.processWxDescFile(fname):
                        pWriteProcessedFile(fname)
            else:
                log.info("{0} does not exist".format(fname))
                
    def processDailyMaxGustFile(self, maxgustfile):
        try:
            gustdata = np.genfromtxt(maxgustfile, dtype=DMAXTYPES, delimiter=",",
                                     usecols=(1,2,3,4,5,6,7,8,9),
                                     autostrip=True, skip_header=1, skip_footer=1)
        except UserWarning:
            log.info("{0} is an empty file".format(maxgustfile))
            return False
        else:
            self.insertMaxGustData(gustdata)
            return True

    def processDailyMaxGustFiles(self):
        maxgustpath = self.config.get("MaxGust", "Path")
        basename = self.config.get("MaxGust", "Basename")
        
        stations = self.getStationIdList()
        for stn in stations:
            fname = pjoin(maxgustpath, basename.format(int(stn)))
            if os.path.exists(fname):
                fstat = flGetStat(fname)
                if (pAlreadyProcessed(fstat[0], fstat[1], 'md5sum', fstat[2]) and
                    self.excludePastProcessed):
                    log.debug("Already processed {0}".format(fname))
                else:
                    if self.processDailyMaxGustFile(fname):
                        pWriteProcessedFile(fname)
            else:
                log.info("{0} does not exist".format(fname))